## Scrape images to extend Dataset

In [1]:
import os 
import json 
import gzip
import pandas as pd 
import urllib.request
import requests
import shutil
import classifier
from imageai.Classification import ImageClassification
from pathlib import Path
from PIL import Image, ImageOps

In [2]:
### Set up image classifier
predictor = ImageClassification()

model_path = "./classifier_models/inception_v3_weights_tf_dim_ordering_tf_kernels.h5"
predictor.setModelTypeAsInceptionV3()
predictor.setModelPath(model_path)
predictor.loadModel()

In [3]:
json_path = r"../AmazonSet/preProcessed_meta_All_Beauty.json"

df_raw = pd.read_json(json_path)

print("Finding image products ...")

def find_second_last(text, pattern):
  return text.rfind(pattern, 0, text.rfind(pattern))


Finding image products ...


In [23]:
#Change for each class
current_class = "Skincare"

#use several key words for small sets. referred from amazon.com
#product_keywords = ["shoe", "sneaker", "boot", "sandal", "slipper"]
#product_keywords = ["sweater", "pullover", "cardigan", "vest", "shrug"]
product_keywords = ["lotion", "creme"]

#current_raw_dir = "raw/"+current_class+"_raw/"
#current_cleaned_dir = "cleaned/"+ current_class+"_cleaned/"
current_dir = os.path.join(".","new", current_class)
#cur_out_txt = "txt/" + current_class+"_duplicates.txt"
#with open(cur_out_txt, "w+") as f:
    #f.close()

In [22]:
current_class = "Watch"
#product_keywords = ["Case", "case", "Bag", "bag"]
#product_keywords = ["Tiara", "tiara", "Crown", "crown"]
#product_keywords = ["moisturizer", "sunscreen", "creme", "Moisturizer", "Sunscreen", "Creme"]
product_keywords = ["Watch"]
current_dir = os.path.join(".","new", current_class)

df = df_raw.copy(deep=True)
df = df[df['image'].map(lambda d: len(d)) > 1]
print(len(df))

frames = []
for keyword in product_keywords:
    df_ = df[df.title.str.contains(keyword)]
    #df_ = df_[df_.title.str.contains("Toothbrush")==False]
    frames.append(df_)
    display(keyword +": "+ str(len(df_)))
df = pd.concat(frames)


11554


'Watch: 45'

In [23]:
image_list = []
for index, row in df.iterrows():
    
    image_list = row['image']
    asin = row['asin']

    if image_list == None:
        continue
    
    for i in range(len(image_list)):
        image_url = image_list[i]
        
        image_url = image_url[:find_second_last(image_url, '.')]+'.jpg'
        image_url = image_url.replace('"', '')
        image_url = image_url.replace('[', '')
        image_url = image_url.replace(' ', '')
        #if 'US40' in image_url:
        #    image_url = image_url.replace( 'US40','SR38%2050')
        #if 'SR38,50' in image_url:
        #    image_url = image_url.replace( 'SR38,50','SR38%2050')
        #if 'SX38_SY50_CR,0,0,38,50' in image_url:
        #    image_url = image_url.replace( 'SX38_SY50_CR,0,0,38,50','SR38%2050')
       
        filename = asin + "_" + str(i) + ".jpg"
        output_path = os.path.join(current_dir, filename)
        Path(current_dir).mkdir(parents=True, exist_ok=True)
        
        # Check if the image was retrieved successfully
        try:
            urllib.request.urlretrieve(image_url, output_path)
            print(index, 'Image successfully downloaded to:', output_path)

            # Check if product image contains an object
            classifier.check_object(predictor, output_path)

        except:
            pass

7184 Image successfully downloaded to: .\new\Watch\B0091NMFAU_0.jpg
7184 Image successfully downloaded to: .\new\Watch\B0091NMFAU_1.jpg
7184 Image successfully downloaded to: .\new\Watch\B0091NMFAU_2.jpg
7791 Image successfully downloaded to: .\new\Watch\B00ANDW3DQ_0.jpg
7791 Image successfully downloaded to: .\new\Watch\B00ANDW3DQ_1.jpg
7791 Image successfully downloaded to: .\new\Watch\B00ANDW3DQ_2.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_0.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_1.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_2.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_3.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_4.jpg
9100 Image successfully downloaded to: .\new\Watch\B00CRY3XP6_5.jpg
9383 Image successfully downloaded to: .\new\Watch\B00DA545W6_0.jpg
9383 Image successfully downloaded to: .\new\Watch\B00DA545W6_1.jpg
9383 Image successfully downloaded to: .\new\Wat

24728 Image successfully downloaded to: .\new\Watch\B016ZV979M_5.jpg
25775 Image successfully downloaded to: .\new\Watch\B018HXM8L0_0.jpg
25775 Image successfully downloaded to: .\new\Watch\B018HXM8L0_1.jpg
25775 Image successfully downloaded to: .\new\Watch\B018HXM8L0_2.jpg
25775 Image successfully downloaded to: .\new\Watch\B018HXM8L0_3.jpg
25775 Image successfully downloaded to: .\new\Watch\B018HXM8L0_4.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_0.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_1.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_2.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_3.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_4.jpg
25778 Image successfully downloaded to: .\new\Watch\B018I1XC0C_5.jpg
25780 Image successfully downloaded to: .\new\Watch\B018I5SGVI_0.jpg
25780 Image successfully downloaded to: .\new\Watch\B018I5SGVI_1.jpg
25780 Image successfully downloade

## Clean up duplicates

In [12]:
import os 
import json 
import gzip
import pandas as pd 
import urllib.request
import requests
import shutil
from PIL import Image, ImageOps
from pathlib import Path
import cv2
import sys

In [7]:
Path(current_raw_dir).mkdir(parents=True, exist_ok=True)
for filename in os.listdir("."):
  if filename.endswith(".jpg"):
    shutil.move("./"+filename, current_raw_dir+filename)

In [33]:
Path(current_cleaned_dir).mkdir(parents=True, exist_ok=True)
for file in os.listdir(current_raw_dir):
    if file.endswith(".jpg"):
        shutil.copy(current_raw_dir+file, current_cleaned_dir+file)

In [9]:
#create a 2d-list containing first-7 id identical products(most cases)

product_dict = {}
def create_pdict(digit):
    product_dict.clear()
    for file in os.listdir(current_cleaned_dir):
        key_id = file[:digit]
        if key_id not in product_dict:
            product_dict.setdefault(key_id,[]).append(file)
        else:
            product_dict[key_id].append(file)
    pdict_len = len(list(product_dict))        
    print("first {} digit: ".format(str(digit)) + str(pdict_len))

In [10]:
def check_duplicates(p_list):
  final_list = []
  index_list = []
  to_write = set()
  for filename in p_list:
    index = filename[11:filename.rfind(".")]
    if index not in index_list:
      final_list.append(filename)
      index_list.append(index)
    else:  
      for f in final_list:
        if index == f[11:f.rfind(".")]:
          img1 = os.path.join(current_cleaned_dir, f)
          img2 = os.path.join(current_cleaned_dir, filename)
          try:
            assert(os.path.isfile(img1))
            assert(os.path.isfile(img2))
          except:
            display(final_list)
            display(index_list)
            display(to_write)
            display(f)
            display(filename)
            
          if open(img1,"rb").read() == open(img2,"rb").read():
            to_write.add(f)
            to_write.add(filename)
            os.remove(img2)
            break
          else:
            continue
      if filename not in to_write:
        final_list.append(filename)

  #for filename in to_write:
    #write_duplicates(filename)
    
def check_duplicates_all(p_list):
    final_list = []
    to_write = set()
    final_list.append(p_list[0])
    for filename in p_list:
        if filename not in final_list:
            for f in final_list:
                img1 = os.path.join(current_cleaned_dir, f)
                img2 = os.path.join(current_cleaned_dir, filename)
                if open(img1,"rb").read() == open(img2,"rb").read():
                    to_write.add(f)
                    to_write.add(filename)
                    os.remove(img2)
                    break
                else:
                    continue
            if filename not in to_write:
                final_list.append(filename)
    #for fn in to_write:
        #write_duplicates(filename)
                    
def write_duplicates(filename):    
  with open(cur_out_txt, 'r+') as f:
  #  line_found = any(filename in line for line in f)
  #  if not line_found:
      f.seek(0, os.SEEK_END)
      f.write(filename[:-4]+"\n")



In [ ]:
digits = [6,4,10]
for digit in digits:
    cnt = 0
    create_pdict(digit)
    pdict_len = len(list(product_dict))  
    print(pdict_len)
    for key in product_dict:
        p_list = product_dict[key]
        if digit == 10:
            if len(p_list) > 1:
                check_duplicates_all(p_list)
        else:        
            asin_list = []
            for file in p_list:
                asin = file[:10]
                if asin not in asin_list:
                    asin_list.append(asin)
            if len(asin_list) > 1:
                check_duplicates(p_list)

        cnt += 1
        sys.stdout.write("\r{0}".format((float(cnt)/pdict_len)*100))
        sys.stdout.flush()
        print('\n')

## Clean all folders in current directory

In [3]:
import os 
import json 
import gzip
import pandas as pd 
import urllib.request
import requests
import shutil
from PIL import Image, ImageOps
from pathlib import Path
import cv2
import sys

In [11]:
#rootdir = r'D:\Github\SeniorDesign\DataSets'
rootdir = r'C:\Users\William Gu\EE Courses\Senior Design\DataSets'
for root, dirs, files in os.walk(rootdir):
    for dir in dirs:
        if dir == 'classifier_models':
            continue
        current_class = dir
        print("Processing Class: " + dir)
        current_cleaned_dir = os.path.join(root, dir)
        
        digits = [6,4,10]
        for digit in digits:
            cnt = 0
            create_pdict(digit)
            pdict_len = len(list(product_dict))  
            print(pdict_len)
            for key in product_dict:
                p_list = product_dict[key]
                if digit == 10:
                    if len(p_list) > 1:
                        check_duplicates_all(p_list)
                else:        
                    asin_list = []
                    for file in p_list:
                        asin = file[:10]
                        if asin not in asin_list:
                            asin_list.append(asin)
                    if len(asin_list) > 1:
                        check_duplicates(p_list)

                cnt += 1
                if cnt%100 == 0:
                    sys.stdout.write("\r{0}".format((float(cnt)/pdict_len)*100))
                    sys.stdout.flush()

Processing Class: Accessories
first 6 digit: 49
49
2.0408163265306123

4.081632653061225

6.122448979591836

8.16326530612245

10.204081632653061

12.244897959183673

14.285714285714285

16.3265306122449

18.367346938775512

20.408163265306122

22.448979591836736

24.489795918367346

26.53061224489796

28.57142857142857

30.612244897959183

32.6530612244898

34.69387755102041

36.734693877551024

38.775510204081634

40.816326530612244

42.857142857142854

44.89795918367347

46.93877551020408

48.97959183673469

51.02040816326531

53.06122448979592

55.10204081632652

57.14285714285714

59.183673469387756

61.224489795918366

63.26530612244898

65.3061224489796

67.3469387755102

69.38775510204081

71.42857142857143

73.46938775510205

75.51020408163265

77.55102040816327

79.59183673469387

81.63265306122449

83.6734693877551

85.71428571428571

87.75510204081633

89.79591836734694

91.83673469387756

93.87755102040816

95.91836734693877

97.95918367346938

100.0

first 4 digit: 7
7
14

10.526315789473683

12.280701754385964

14.035087719298245

15.789473684210526

17.543859649122805

19.298245614035086

21.052631578947366

22.807017543859647

24.561403508771928

26.31578947368421

28.07017543859649

29.82456140350877

31.57894736842105

33.33333333333333

35.08771929824561

36.84210526315789

38.59649122807017

40.35087719298245

42.10526315789473

43.859649122807014

45.614035087719294

47.368421052631575

49.122807017543856

50.877192982456144

52.63157894736842

54.385964912280706

56.14035087719298

57.89473684210527

59.64912280701754

61.40350877192983

63.1578947368421

64.91228070175438

66.66666666666666

68.42105263157895

70.17543859649122

71.9298245614035

73.68421052631578

75.43859649122807

77.19298245614034

78.94736842105263

80.7017543859649

82.45614035087719

84.21052631578947

85.96491228070175

87.71929824561403

89.47368421052632

91.22807017543859

92.98245614035088

94.73684210526315

96.49122807017544

98.24561403508771

100.0

first 10 dig

21.69811320754717

21.77672955974843

21.855345911949687

21.933962264150946

22.0125786163522

22.09119496855346

22.169811320754718

22.248427672955977

22.32704402515723

22.40566037735849

22.48427672955975

22.562893081761008

22.641509433962266

22.72012578616352

22.79874213836478

22.87735849056604

22.955974842767297

23.034591194968552

23.11320754716981

23.19182389937107

23.270440251572328

23.349056603773587

23.427672955974842

23.5062893081761

23.58490566037736

23.663522012578618

23.742138364779876

23.82075471698113

23.89937106918239

23.97798742138365

24.056603773584907

24.135220125786162

24.21383647798742

24.29245283018868

24.371069182389938

24.449685534591197

24.528301886792452

24.60691823899371

24.68553459119497

24.764150943396228

24.842767295597483

24.92138364779874

25.0

25.078616352201262

25.157232704402517

25.235849056603776

25.31446540880503

25.393081761006293

25.471698113207548

25.550314465408807

25.628930817610062

25.707547169811324


89.15094339622641

89.22955974842768

89.30817610062893

89.38679245283019

89.46540880503144

89.54402515723271

89.62264150943396

89.70125786163521

89.77987421383648

89.85849056603774

89.937106918239

90.01572327044025

90.09433962264151

90.17295597484278

90.25157232704403

90.33018867924528

90.40880503144653

90.4874213836478

90.56603773584906

90.64465408805032

90.72327044025157

90.80188679245283

90.88050314465409

90.95911949685535

91.0377358490566

91.11635220125787

91.19496855345912

91.27358490566037

91.35220125786164

91.4308176100629

91.50943396226415

91.5880503144654

91.66666666666666

91.74528301886792

91.82389937106919

91.90251572327044

91.98113207547169

92.05974842767296

92.13836477987421

92.21698113207547

92.29559748427673

92.37421383647799

92.45283018867924

92.5314465408805

92.61006289308176

92.68867924528303

92.76729559748428

92.84591194968553

92.9245283018868

93.00314465408806

93.08176100628931

93.16037735849056

93.23899371069182

9

36.78551216751556

36.84210526315789

36.898698358800225

36.95529145444256

37.011884550084886

37.06847764572722

37.125070741369555

37.18166383701188

37.23825693265422

37.29485002829655

37.351443123938886

37.408036219581206

37.46462931522354

37.521222410865875

37.5778155065082

37.634408602150536

37.69100169779287

37.747594793435205

37.80418788907753

37.86078098471987

37.9173740803622

37.97396717600453

38.030560271646856

38.08715336728919

38.143746462931524

38.20033955857385

38.256932654216186

38.31352574985852

38.37011884550085

38.42671194114318

38.48330503678552

38.539898132427844

38.59649122807017

38.653084323712505

38.70967741935484

38.76627051499717

38.8228636106395

38.879456706281836

38.93604980192416

38.9926428975665

39.04923599320883

39.105829088851166

39.16242218449349

39.21901528013582

39.275608375778155

39.33220147142048

39.38879456706282

39.44538766270515

39.501980758347486

39.55857385398981

39.61516694963215

39.67176004527448


...

16.328371673432567

16.373477672530445

16.418583671628326

16.463689670726207

16.508795669824085

16.553901668921966

16.599007668019848

16.644113667117725

16.689219666215607

16.734325665313488

16.77943166441137

16.824537663509247

16.869643662607125

16.914749661705006

16.959855660802887

17.004961659900765

17.050067658998646

17.095173658096527

17.14027965719441

17.185385656292286

17.230491655390168

17.27559765448805

17.320703653585927

17.365809652683804

17.410915651781686

17.456021650879567

17.501127649977448

17.546233649075326

17.591339648173207

17.63644564727109

17.681551646368966

17.726657645466847

17.77176364456473

17.81686964366261

17.861975642760484

17.907081641858365

17.952187640956247

17.997293640054128

18.042399639152006

18.087505638249887

18.132611637347768

18.17771763644565

18.222823635543527

18.26792963464141

18.31303563373929

18.358141632837167

18.403247631935045

18.448353631032926

18.493459630130808

18.538565629228685

18.583671

54.53315290933695

54.57825890843482

54.623364907532704

54.66847090663058

54.713576905728466

54.75868290482634

54.80378890392422

54.84889490302211

54.89400090211998

54.93910690121786

54.98421290031574

55.029318899413624

55.074424898511495

55.11953089760938

55.164636896707265

55.20974289580515

55.25484889490302

55.299954894000905

55.34506089309878

55.39016689219666

55.43527289129454

55.48037889039242

55.52548488949031

55.57059088858818

55.61569688768606

55.66080288678394

55.705908885881826

55.751014884979696

55.79612088407758

55.841226883175466

55.88633288227334

55.93143888137122

55.976544880469106

56.021650879566984

56.06675687866486

56.11186287776274

56.156968876860624

56.20207487595851

56.24718087505638

56.292286874154264

56.33739287325214

56.38249887235002

56.4276048714479

56.47271087054578

56.51781686964367

56.56292286874154

56.60802886783942

56.65313486693731

56.698240866035185

56.74334686513306

56.78845286423094

56.833558863328825

93.50473612990527

93.54984212900315

93.59494812810104

93.64005412719891

93.6851601262968

93.73026612539468

93.77537212449256

93.82047812359043

93.86558412268832

93.9106901217862

93.95579612088409

94.00090211998196

94.04600811907984

94.09111411817773

94.1362201172756

94.18132611637347

94.22643211547135

94.27153811456924

94.31664411366711

94.361750112765

94.40685611186288

94.45196211096076

94.49706811005863

94.54217410915652

94.5872801082544

94.63238610735227

94.67749210645016

94.72259810554804

94.76770410464593

94.8128101037438

94.85791610284167

94.90302210193956

94.94812810103744

94.99323410013531

95.0383400992332

95.08344609833108

95.12855209742895

95.17365809652684

95.21876409562472

95.2638700947226

95.30897609382048

95.35408209291836

95.39918809201625

95.44429409111413

95.489400090212

95.53450608930987

95.57961208840776

95.62471808750563

95.66982408660351

95.7149300857014

95.76003608479928

95.80514208389715

95.85024808299504

95.89

60.207612456747405

60.32295271049596

60.438292964244525

60.553633217993074

60.66897347174164

60.78431372549019

60.89965397923876

61.014994232987306

61.13033448673587

61.245674740484425

61.36101499423299

61.47635524798154

61.5916955017301

61.70703575547867

61.82237600922722

61.93771626297578

62.05305651672434

62.1683967704729

62.28373702422145

62.399077277970015

62.51441753171857

62.629757785467135

62.745098039215684

62.86043829296425

62.9757785467128

63.091118800461366

63.206459054209915

63.32179930795848

63.437139561707035

63.5524798154556

63.667820069204154

63.78316032295272

63.89850057670127

64.01384083044984

64.12918108419838

64.24452133794695

64.3598615916955

64.47520184544406

64.59054209919262

64.70588235294117

64.82122260668973

64.9365628604383

65.05190311418684

65.16724336793541

65.28258362168397

65.39792387543253

65.51326412918108

65.62860438292965

65.7439446366782

65.85928489042676

65.97462514417532

66.08996539792388

66.2053

KeyboardInterrupt: 

## Create Difficult sets

In [50]:
import os 
import json 
import gzip
import pandas as pd 
import urllib.request
import requests
import shutil
from PIL import Image, ImageOps
from pathlib import Path
import cv2
import sys
import copy
from distutils.dir_util import copy_tree

In [18]:
#rootdir = r'D:\Github\SeniorDesign\DataSets'
rootdir = r'C:\Users\William Gu\EE Courses\Senior Design\DataSets'
for root, dirs, files in os.walk(rootdir):
    for dir in dirs:
        if dir == 'classifier_models':
            continue
        current_class = dir
        print("Processing Class: " + dir)
        cur_dir = os.path.join(root, dir)
        cur_dir_copy = os.path.join('./newDataSets', dir)
        Path(cur_dir_copy).mkdir(parents=True, exist_ok=True)

Processing Class: Accessories
Processing Class: Backpacks
Processing Class: Bags
Processing Class: Belts
Processing Class: Blouses
Processing Class: Dresses
Processing Class: Glasses
Processing Class: Gloves
Processing Class: Hats
Processing Class: Jackets
Processing Class: Jeans
Processing Class: Jewelry
Processing Class: Luggages
Processing Class: Neckties
Processing Class: Pants
Processing Class: Scarves
Processing Class: Shirts
Processing Class: Shoes
Processing Class: Shorts
Processing Class: Sleepwear
Processing Class: Socks
Processing Class: Suits
Processing Class: Sweaters
Processing Class: Umbrellas
Processing Class: underwear
Processing Class: Wallets
Processing Class: Watches


In [73]:
def create_pdict(digit, target_dir):
    global product_dict
    product_dict.clear()
    for file in os.listdir(target_dir):
        key_id = file[:digit]
        if key_id not in product_dict:
            product_dict.setdefault(key_id,[]).append(file)
        else:
            product_dict[key_id].append(file)
    pdict_len = len(product_dict)        
    print("first {} digit have: ".format(str(digit)) + str(pdict_len)+ " keys.")

def process_dict(digit):
    new_dict = {}
    global product_dict
    for key in product_dict:
        #img_list = product_dict.pop(key)
        img_list = product_dict[key]
        new_key = key[:digit]
        if new_key not in new_dict:
            new_dict.setdefault(new_key,[])
        new_dict[new_key] += img_list
        
    product_dict = copy.deepcopy(new_dict)    
    



In [74]:
#hard_dir = r'C:\Users\William Gu\EE Courses\Senior Design\Glasses_hard'
#cur_dir = r'C:\Users\William Gu\EE Courses\Senior Design\Glasses'

product_dict = {}
#create_pdict(9)
digits = [8, 7, 6]

rootdir = r'C:\Users\William Gu\EE Courses\Senior Design\DataSets'
for root, dirs, files in os.walk(rootdir):
    for dir in dirs:
        if dir == 'classifier_models':
            continue
        current_class = dir
        print("Processing Class: " + dir)
        cur_dir = os.path.join(root, dir)
        cur_dir_copy = os.path.join('./newDataSets', dir)
        Path(cur_dir_copy).mkdir(parents=True, exist_ok=True)
        
        create_pdict(9, cur_dir)
        for digit in digits:
            process_dict(digit)
        
        for key in product_dict:
            p_list = product_dict[key]
            key_path = os.path.join(cur_dir_copy, key)
            Path(key_path).mkdir(parents=True, exist_ok=True)
            for file in p_list:
                shutil.copy(os.path.join(cur_dir, file), os.path.join(key_path, file))


Processing Class: Accessories
first 9 digit have: 88 keys.
Processing Class: Backpacks
first 9 digit have: 38 keys.
Processing Class: Bags
first 9 digit have: 29 keys.
Processing Class: Belts
first 9 digit have: 95 keys.
Processing Class: Blouses
first 9 digit have: 7 keys.
Processing Class: Dresses
first 9 digit have: 519 keys.
Processing Class: Glasses
first 9 digit have: 1761 keys.
Processing Class: Gloves
first 9 digit have: 35 keys.
Processing Class: Hats
first 9 digit have: 104 keys.
Processing Class: Jackets
first 9 digit have: 69 keys.
Processing Class: Jeans
first 9 digit have: 85 keys.
Processing Class: Jewelry
first 9 digit have: 764 keys.
Processing Class: Luggages
first 9 digit have: 20 keys.
Processing Class: Neckties
first 9 digit have: 30 keys.
Processing Class: Pants
first 9 digit have: 104 keys.
Processing Class: Scarves
first 9 digit have: 9 keys.
Processing Class: Shirts
first 9 digit have: 2200 keys.
Processing Class: Shoes
first 9 digit have: 860 keys.
Processing 

In [25]:
dict = {1:[1,2], 2:[1,2,3]}
print(dict)
a = dict.pop(1,None)
print(a)
print(dict)

{1: [1, 2], 2: [1, 2, 3]}
[1, 2]
{2: [1, 2, 3]}
